# Jupyter Notebooklets Demo

### [@ianhellen](https://twitter.com/ianhellen)
#### Principal Dev - MSTIC, Azure Security

# What are notebooklets?

Collections of notebook cells that implement some useful reusable sequence

## Rationale
- Notebook code can quickly become complex and length:
  - Can obscure the information you are trying to display
  - Can be intimidating to non-developers
- Notebook code cells are not easily re-useable:
  - You can copy and paste but how do you sync changes back to original notebook?
  - Difficult to discover code snippets in notebooks
- Notebook code is often fragile:
  - Often not parameterized
  - Code blocks are frequently dependent on global values assigned earlier
  - Output data is not in any standard format
  - Difficult to test

## Characteristics of Notebooklets
- One or small number of entry points
- Must be paramertizable (e.g. you can supply hostname, IP Address, time range, etc.)
- Can query, process or visualize data (or any combination)
- Typically return a result or package of results for use later in the notebook


---
# Initializing the Notebook
Notebooklets depend on msticpy so we import/initialize this package.

In [6]:
import sys
import os
from IPython.display import display, HTML, Markdown

from msticpy.nbtools.nbinit import init_notebook
init_notebook(namespace=globals());

Processing imports....
Checking configuration....
No errors found.

The following configuration warnings were found: 
 -------------------------------------------------
No AzureCLI section in settings.
Setting options....


---
# Notebooklets in use

## Import the package
- Discovers and imports notebooklet classes/modules

In [2]:
# pip install git+https://github.com/microsoft/msticnb

In [3]:
import msticnb as nb

7 notebooklets loaded.


---
## Calling init()
Before using any of the notebooklets you need to initialize the providers.

Providers are the libraries that do the work of fetching data from external sources that are
then used by the notebooklet code.

init() does the following:
- Loads required data providers
- Authenticates to providers if required at startup
- Can supply list of providers to load
- Can pass parameters to each provider (settings loaded from config by default)

In [4]:
nb.init?

Signature:
nb.init(
    query_provider:str='LogAnalytics',
    providers:Union[List[str], NoneType]=None,
    **kwargs,
)
Docstring:
Instantiate an instance of DataProviders.

Parameters
----------
query_provider : str, optional
    DataEnvironment name of the primary query provider.
    You can add addtional query providers by including them
    in the `providers` list.
providers : Optional[List[str]], optional
    A list of provider names, by default "LogAnalytics"

Other Parameters
----------------
kwargs
    You can pass parameters to individual providers using
    the following notation:
    `ProviderName_param_name="param_value"
    Where `ProviderName` is the name of the data provider,
    `param_name` is the parameter name expected by the
    provider and `param_value` is the value to assign to
    `param_name`. `param_value` can be any type.

    Depending on the provider, these parameters (with the
    prefix stripped) are sent to either the constructor or
    `connect` metho

### Available Providers

In [5]:
nb.DataProviders.list_providers()

['LogAnalytics',
 'AzureSentinel',
 'Kusto',
 'AzureSecurityCenter',
 'SecurityGraph',
 'MDATP',
 'LocalData',
 'Splunk',
 'tilookup',
 'geolitelookup',
 'ipstacklookup']

### Default Providers

In [7]:
nb.DataProviders.get_def_providers()

['tilookup', 'geolitelookup']

### Running init, adding ipstacklookup to the default set of providers.
You can also prefix a provider name with "-" to remove it from the default set.

You can also specify an explicit list of providers to override the defaults entirely. E.g
```
nb.init(query_provider="AzureSentinel", providers=["ipstacklookup", "tilookup"])
```

> **Note** you cannot mix the "+"/"-" with un-prefixed provider names.
> Doing this will cause an error to be thrown.
> e.g. <br>
> `nb.init(query_provider="AzureSentinel", providers=["+ipstacklookup", "tilookup"])`
> <br>is illegal.

In [8]:
nb.init(query_provider="LogAnalytics", providers=["-tilookup", "+ipstacklookup"])

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Loaded providers: LogAnalytics, geolitelookup, ipstacklookup


---
## Using LocalData Provider
The LocalData provider allows you to substitue local files for queries that you normally
make to online data sources such as AzureSentinel.

When we call init() we use the "LocalData_" prefix to pass the "query_paths" and "data_paths"
parameters to the underlying provider.

- Specify a folder where data files are stored with `LocalData_data_paths` (list[str])
- Specify a folder containing query definition files `LocalData_query_paths` (list[str])


In notebooklets queries are available as self.query_provider

In [9]:
nb.init(
    "LocalData", providers=["-tilookup"],
    LocalData_data_paths=["/src/msticnb/tests/testdata"],
    LocalData_query_paths=["/src/msticnb/tests/testdata"],
)

\src\msticnb\tests\testdata\msticpyconfig-test.yaml is not a valid query definition file - skipping.
\src\msticnb\tests\testdata\custom_nb\host\host_test_nb.yaml is not a valid query definition file - skipping.
Loaded providers: LocalData, geolitelookup


---
## Notebooklet classes are discovered and imported at load time

Although you can manually initiate a a run to read more notebooklets.

#### The `nblts` attribute exposes notebooklets (niblets?) in a tree structure
- Useful for autocomplete when you more or less know what you're looking for

The top level in the hierarchy is the data environment (e.g. azsent == AzureSentinel). Beneath these the notebooklets are grouped into various categories such as host, network, etc.

In [10]:
print(nb.nblts)

azsent
  account
    AccountSummary (Notebooklet)
  alert
    EnrichAlerts (Notebooklet)
  host
    HostLogonsSummary (Notebooklet)
    HostSummary (Notebooklet)
    WinHostEvents (Notebooklet)
  network
    NetworkFlowSummary (Notebooklet)
template
  TemplateNB (Notebooklet)



Access an individual notebook using this path structure

In [11]:
nb.nblts.azsent.host.HostSummary?

Init signature:
nb.nblts.azsent.host.HostSummary(
    data_providers:Union[<msticnb.data_providers.SingletonDecorator object at 0x0000027434106828>, NoneType]=None,
    **kwargs,
)
Docstring:     
HostSummary Notebooklet class.

Queries and displays information about a host including:

- IP address assignment
- Related alerts
- Related hunting/investigation bookmarks
- Azure subscription/resource data.


Default Options
---------------
- heartbeat: Query Heartbeat table for host information.
- azure_net: Query AzureNetworkAnalytics table for host network topology information.
- alerts: Query any alerts for the host.
- bookmarks: Query any bookmarks for the host.
- azure_api: Query Azure API for VM information.

Other Options
-------------
None
Init docstring:
Intialize a new instance of the notebooklet class.

Parameters
----------
data_providers : DataProviders, Optional
    Optional DataProviders instance to query data.
    Most classes require this.

Raises
------
MsticnbDataProvide

### Notebooklets are exposed in `nb.nb_index`
The values reflect the physical path in which the notebooklets are stored (you can ignore this)

In [12]:
nb.nb_index

{'nblts.azsent.account.AccountSummary': msticnb.nb.azsent.account.account_summary.AccountSummary,
 'nblts.azsent.alert.EnrichAlerts': msticnb.nb.azsent.alert.ti_enrich.EnrichAlerts,
 'nblts.azsent.host.HostLogonsSummary': msticnb.nb.azsent.host.host_logons_summary.HostLogonsSummary,
 'nblts.azsent.host.HostSummary': msticnb.nb.azsent.host.host_summary.HostSummary,
 'nblts.azsent.host.WinHostEvents': msticnb.nb.azsent.host.win_host_events.WinHostEvents,
 'nblts.azsent.network.NetworkFlowSummary': msticnb.nb.azsent.network.network_flow_summary.NetworkFlowSummary,
 'nblts.template.TemplateNB': msticnb.nb.template.nb_template.TemplateNB}

## There is a find function that looks for:
- text or regulate expressions
- searches class docstring
- metadata such as entities supported and options supported

In [13]:
nb.find("host, net.*", full_match=True)

[('HostSummary', msticnb.nb.azsent.host.host_summary.HostSummary),
 ('NetworkFlowSummary',
  msticnb.nb.azsent.network.network_flow_summary.NetworkFlowSummary)]

---
# More detailed (and user-friendly) help in the `show_help()` method

In [14]:
nb.nblts.azsent.host.HostSummary.show_help()

---
# How are notebooklets used?

## Most require time range parameters

Usually the notebooklet also the ID of the entity that you're running the notebooklet for. For example, a host name, an IP Address, etc.

Some notebooklets process data in the form of a dataframe. Use the `data` parameter to pass this.

> **Note** You can also pass other parameters used by the notebooklet as keyword arguments (`**kwargs`)

In [15]:
time_span = nbwidgets.QueryTime(auto_display=True, units="day", origin_time=pd.to_datetime("2019-02-10"), before=10)
from msticnb.common import TimeSpan

HTML(value='<h4>Set query time boundaries</h4>')

## Run the notebooklet using the `run()` method

>  **Note:** You'll want to assign the return value of `run()` to something or terminate with a semicolon<br>
>  Both the notebooklet and the return `result` class generate displayable output - so you'll get
>  a lot of duplicated output.

In [16]:
host_summary = nb.nblts.azsent.host.HostSummary()
host_sum_rslt = host_summary.run(value="Msticalertswin1", timespan=time_span)

msticpy.common.exceptions.MsticpyAzureConfigError('No AzureCLI section found in configuration settings.')

Host not found: Msticalertswin1


{'AdditionalData': {}, 'HostName': 'Msticalertswin1', 'Type': 'host'}


Loading BokehJS ...

Getting data from Bookmarks...


## Result classes content can be displayed in the notebook
Use `display(result)` if you want to display the content in the middle of a cell

In [17]:
host_sum_rslt

HostSummaryResult(description='Host summary', timespan=<msticpy.nbtools.nbwidgets.QueryTime object at 0x0000027434647630>, _attribute_desc={'host_entity': ('msticpy.data.nbtools.entities.Host', 'The host entity object contains data about the host such as name, environment, operating system version, IP addresses and Azure VM details. Depending on the type of host, not all of this data may be populated.'), 'related_alerts': ('pd.DataFrame', 'Pandas DataFrame of any alerts recorded for the host within the query time span.'), 'alert_timeline': ('', 'Bokeh time plot of alerts recorded for host.'), 'related_bookmarks': ('pd.DataFrame', 'Pandas DataFrame of any investigation bookmarks relating to the host.'), 'timespan': ('TimeSpan', 'Time span for the queried results data.')}, host_entity=Host(Type=host, HostName=Msticalertswin1), related_alerts=                                TenantId       TimeGenerated  \
0   52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:29:07   
1   52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 01:59:09   
2   52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:29:07   
3   52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:43:27   
4   52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 01:54:11   
5   52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:19:10   
6   52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 01:49:11   
7   52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:24:07   
8   52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:24:07   
9   52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:29:07   
10  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:24:07   
11  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:09:07   
12  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:04:11   
13  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 02:14:08   
14  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-18 03:19:29   
15  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-16 23:49:09   
16  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-16 23:54:11   
17  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-17 00:09:10   
18  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-16 22:04:10   
19  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-16 21:30:06   
20  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-16 03:24:04   
21  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-15 03:51:09   
22  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-15 04:03:30   
23  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-15 10:14:37   
24  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-15 20:27:38   

                                        AlertDisplayName  \
0                                 SSH Anomalous Login ML   
1                                 SSH Anomalous Login ML   
2                                 SSH Anomalous Login ML   
3                                 SSH Anomalous Login ML   
4                                 SSH Anomalous Login ML   
5                                 SSH Anomalous Login ML   
6                                 SSH Anomalous Login ML   
7                                 SSH Anomalous Login ML   
8                                 SSH Anomalous Login ML   
9                                 SSH Anomalous Login ML   
10                                SSH Anomalous Login ML   
11                                SSH Anomalous Login ML   
12                                SSH Anomalous Login ML   
13                                SSH Anomalous Login ML   
14                                SSH Anomalous Login ML   
15                                SSH Anomalous Login ML   
16                                SSH Anomalous Login ML   
17                                SSH Anomalous Login ML   
18                                SSH Anomalous Login ML   
19                                SSH Anomalous Login ML   
20                     Detected suspicious file download   
21                     Detected suspicious file download   
22  Possible suspicious scheduling tasks access detected   
23                            Security inci

---
## Simple Notebooklet browser

In [18]:
nb.browse()

In [19]:
# value="MSTICAlertsWin1", timespan=time_span

win_host_events = nb.nblts.azsent.host.WinHostEvents()
timespan = TimeSpan(start="2020-05-07 00:10")
win_host_events_rslt = win_host_events.run(value="MSTICAlertsWin1", timespan=timespan)

Getting data from SecurityEvent...


Activity,DWM-1,DWM-2,IUSR,LOCAL SERVICE,MSTICAdmin,MSTICAlertsWin1$,NETWORK SERVICE,No Account,SYSTEM,ian
1100 - The event logging service has shut down.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
4608 - Windows is starting up.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
4616 - The system time was changed.,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4625 - An account failed to log on.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
4634 - An account was logged off.,0.000000,4.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,2.000000
4647 - User initiated logoff.,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4648 - A logon was attempted using explicit credentials.,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000
4672 - Special privileges assigned to new logon.,2.000000,2.000000,1.000000,1.000000,14.000000,0.000000,1.000000,0.000000,60.000000,0.000000
4720 - A user account was created.,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4722 - A user account was enabled.,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Activity,MSTICAdmin
4720 - A user account was created.,2
4722 - A user account was enabled.,2
4724 - An attempt was made to reset an account's password.,4
4726 - A user account was deleted.,2
4728 - A member was added to a security-enabled global group.,2
4729 - A member was removed from a security-enabled global group.,2
4732 - A member was added to a security-enabled local group.,4
4733 - A member was removed from a security-enabled local group.,3
4738 - A user account was changed.,5


Loading BokehJS ...

## Additional operations apart from `run()`
We can use expand events to unpack the `EventData` column for selected EventIDs

In [20]:
win_host_events_rslt.account_events.head(5)

,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,PartitionKey,RowKey,StorageAccount,AzureDeploymentID,AzureTableName,AccessList,AccessMask,AccessReason,AccountDomain,AccountExpires,AccountName,AccountSessionIdentifier,...,TargetUserSid,TemplateContent,TemplateDSObjectFQDN,TemplateInternalName,TemplateOID,TemplateSchemaVersion,TemplateVersion,TokenElevationType,TransmittedServices,UserAccountControl,UserParameters,UserPrincipalName,UserWorkstations,VirtualAccount,VendorIds,Workstation,WorkstationName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,Type,_ResourceId,EventProperties
47,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.173,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,27df6071-1e81-4e24-934c-dc96667b83ab,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'MemberName': '-', 'MemberSid': 'S-1-5-21-996632719-2361334927-4038480536-1118', 'TargetUserNam..."
48,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.173,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4720,4720 - A user account was created.,,,,,,,,,,%%1794,,,...,,,,,,,,,,\t\t%%2080 \t\t%%2082 \t\t%%2084,%%1793,-,%%1793,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,2c09036a-5ca7-4115-9ddf-e9eb49c14247,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'abai$', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S-1-5-21-996632..."
49,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.183,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4722,4722 - A user account was enabled.,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,fefd6761-e431-4cfa-9cd2-c5700f6186df,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'TargetUserName': 'abai$', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S-1-5-21-996632..."
50,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.183,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Dummy...",4738,4738 - A user account was changed.,,,,,,,,,,%%1794,,,...,,,,,,,,,,\t\t%%2048 \t\t%%2050,-,-,%%1793,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,1d3997a3-9ede-4f9b-877a-eaabc63a3c1e,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...,"{'Dummy': '-', 'TargetUserName': 'abai$', 'TargetDomainName': 'MSTICAlertsWin1', 'TargetSid': 'S..."
51,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.183,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.c

In [21]:
win_host_events.expand_events(event_ids=4728).head(5)

,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,MemberName,MemberSid,PrivilegeList,SubjectAccount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TargetAccount,TargetDomainName,TargetSid,TargetUserName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,Type,_ResourceId
47,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.173,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,-,S-1-5-21-996632719-2361334927-4038480536-1118,-,MSTICAlertsWin1\MSTICAdmin,MSTICAlertsWin1,0xbd57571,MSTICAdmin,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAlertsWin1\None,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-513,None,263a788b-6526-4cdc-8ed9-d79402fe4aa0,27df6071-1e81-4e24-934c-dc96667b83ab,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
58,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.447,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,-,S-1-5-21-996632719-2361334927-4038480536-1119,-,MSTICAlertsWin1\MSTICAdmin,MSTICAlertsWin1,0xbd57571,MSTICAdmin,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAlertsWin1\None,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-513,None,263a788b-6526-4cdc-8ed9-d79402fe4aa0,73b0fe4e-9886-43ab-afa6-b43eb7434402,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...


---
# Anatomy of a Notebooklet

# Three sections:
- Results class - what is it going to return
- Notebooklet class - `run()` defines what the notebooklet does
- Code - series of functions that do the actual work

In [22]:
nb.nblts.template.TemplateNB.import_cell()

In [ ]:
# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See License.txt in the project root for
# license information.
# --------------------------------------------------------------------------
"""
Template notebooklet.

Notebooklet modules have three main sections:
- Result class definition
  This defines the attributes and descriptions of the data that you
  want to return from the notebooklet.
- Notebooklet class definition
  This is the entry point for running the notebooklet. At minimum
  it should be a class derived from Notebooklet that implements
  a `run` method and returns your result class.
- Functions
  These do most of the work of the notebooklet and usually the code
  that is copied from or adapted from the original notebook.

Having the latter section is optional. You can choose to implement
this functionality in instance methods of the notebooklet class.

However, there are advantages to keeping these as separate functions
outside the class. It means that all the data used in the functions
has to be passed around as parameters and return values. This can
improve the clarity of the code and reduce errors due to some
dependency on some mysterious global state.

If the user of your notebooklet wants to import the module's code
into a notebook to read and possibly adapt it, having standalone
functions will make it easier from them understand and work with
the code.

"""
from typing import Any, Optional, Iterable, Union, Dict

import attr
from bokeh.plotting.figure import Figure
import pandas as pd
from msticpy.nbtools import nbdisplay

# Note - when moved to the final location (e.g.
# nb/environ/category/mynotebooklet.py)
# you will need to change the "msticnb." to "msticnb." in these
# imports because the relative path has changed.
from msticnb.common import (
    TimeSpan,
    MsticnbMissingParameterError,
    nb_data_wait,
    nb_print,
    set_text,
    nb_markdown,
)

# change the "msticnb." to "msticnb."
from msticnb.notebooklet import Notebooklet, NotebookletResult, NBMetadata
from msticnb. import nb_metadata

# change the ".." to "msticnb."
from msticnb._version import VERSION

__version__ = VERSION
__author__ = "Your name"


# Read module metadata from YAML
_CLS_METADATA: NBMetadata
_CELL_DOCS: Dict[str, Any]
_CELL_DOCS = {'run': {'title': 'Title for the run method (main title)', 'hd_level': 1, 'text': 'Write your introductory text here\nData and plots are stored in the result class returned by this function.\nIf you use **markdown** syntax in this block add the following to use markdown processing.', 'md': True}, 'display_event_timeline': {'title': 'Display the timeline.', 'text': ' This may take some time to complete for large numbers of events.\nIt will do: - Item one - Item two\nSince some groups will be undefined these can show up as `NaN`.\nNote: use a quoted string if you want to include yaml reserved chars such as ":" ', 'md': True}}

_CLS_METADATA = nb_metadata.NBMetadata(name='TemplateNB', mod_name='msticnb.nb.template.nb_template', description='Template YAML for Notebooklet', default_options=[{'all_events': 'Gets all events about blah'}, {'plot_events': 'Display and summary and timeline of events.'}], other_options=[{'get_metadata': 'fetches additional metadata about the entity'}], entity_types=['host'], keywords=['host', 'computer', 'heartbeat', 'windows', 'account'], req_providers=['AzureSentinel|LocalData', 'tilookup'])


# pylint: disable=too-few-public-methods
# Rename this class
@attr.s(auto_attribs=True)
class TemplateResult(NotebookletResult):
    """
    Template Results.

    Attributes
    ----------
    all_events : pd.DataFrame
        DataFrame of all raw events retrieved.
    plot : bokeh.models.LayoutDOM
        Bokeh plot figure showing the account events on an
        interactive timeline.
    additional_info: dict
        Additional information for my notebooklet.

    """

    description: str = "Windows Host Security Events"

    # Add attributes as needed here.
    # Make sure they are documented in the Attributes section
    # above.
    all_events: pd.DataFrame = None
    plot: Figure = None
    additional_info: Optional[dict] = None


# pylint: enable=too-few-public-methods


# Rename this class
class TemplateNB(Notebooklet):
    """
    Template Notebooklet class.

    Detailed description of things this notebooklet does:

    - Fetches all events from XYZ
    - Plots interesting stuff
    - Returns extended metadata about the thing

    Document the options that the Notebooklet takes, if any,
    Use these control which parts of the notebooklet get run.

    """

    # assign metadata from YAML to class variable
    metadata = _CLS_METADATA
    __doc__ = nb_metadata.update_class_doc(__doc__, metadata)
    _cell_docs = _CELL_DOCS

    # @set_text decorator will display the title and text every time
    # this method is run.
    # The key value refers to an entry in the `output` section of
    # the notebooklet yaml file.
    @set_text(docs=_CELL_DOCS, key="run")
    def run(
        self,
        value: Any = None,
        data: Optional[pd.DataFrame] = None,
        timespan: Optional[TimeSpan] = None,
        options: Optional[Iterable[str]] = None,
        **kwargs,
    ) -> TemplateResult:
        """
        Return XYZ summary.

        Parameters
        ----------
        value : str
            Host name - The key for searches - e.g. host, account, IPaddress
        data : Optional[pd.DataFrame], optional
            Alternatively use a DataFrame as input.
        timespan : TimeSpan
            Timespan for queries
        options : Optional[Iterable[str]], optional
            List of options to use, by default None.
            A value of None means use default options.
            Options prefixed with "+" will be added to the default options.
            To see the list of available options type `help(cls)` where
            "cls" is the notebooklet class or an instance of this class.

        Returns
        -------
        TemplateResult
            Result object with attributes for each result type.

        Raises
        ------
        MsticnbMissingParameterError
            If required parameters are missing

        """
        # This line use logic in the superclass to populate options
        # (including default options) into this class.
        super().run(
            value=value, data=data, timespan=timespan, options=options, **kwargs
        )

        if not value:
            raise MsticnbMissingParameterError("value")
        if not timespan:
            raise MsticnbMissingParameterError("timespan.")

        # Create a result class
        result = TemplateResult()
        result.description = self.metadata.description
        result.timespan = timespan

        # You might want to always do some tasks irrespective of
        # options sent
        all_events_df = _get_all_events(
            self.query_provider, host_name=value, timespan=timespan
        )
        result.all_events = all_events_df

        if "plot_events" in self.options:
            result.plot = _display_event_timeline(acct_event_data=all_events_df)

        if "get_metadata" in self.options:
            result.additional_info = _get_metadata(host_name=value, timespan=timespan)

        # Assign the result to the _last_result attribute
        # so that you can get to it without having to re-run the operation
        self._last_result = result  # pylint: disable=attribute-defined-outside-init

        return self._last_result

    # You can add further methods to do things after (or before) the main
    # run method. You might need these if you want to add an interaction
    # point where the user needs to select and option. For example, you
    # could have a "select_account" method that uses a widget to let the
    # notebook user pick from a list. Then have a follow on method that
    # does something with this choice.
    def run_additional_operation(
        self, event_ids: Optional[Union[int, Iterable[int]]] = None
    ) -> pd.DataFrame:
        """
        Addition method.

        Parameters
        ----------
        event_ids : Optional[Union[int, Iterable[int]]], optional
            Single or interable of event IDs (ints).

        Returns
        -------
        pd.DataFrame
            Results with expanded columns.

        """
        # Include this to check the "run()" has happened before this method
        # can be run
        if (
            not self._last_result or self._last_result.all_events is None
        ):  # type: ignore
            print(
                "Please use 'run()' to fetch the data before using this method.",
                "\nThen call 'expand_events()'",
            )
            return None
        # Print a status message - this will not be displayed if
        # the user has set the global "verbose" option to False.
        nb_print("We maybe about to wait some time")

        nb_markdown("Print some message that always displays", "blue, bold")
        return _do_additional_thing(
            evt_df=self._last_result.all_events,  # type: ignore
            event_ids=event_ids,
        )
        # Note you can also assign new items to the result class in
        # self._last_result and return the updated result class.


# This section contains functions that do the work. It can be split into
# cells recognized by some editors (like VSCode) but this is optional

# %%
# Get Windows Security Events
def _get_all_events(qry_prov, host_name, timespan):
    nb_data_wait("SecurityEvent")

    # Tell the user that you're fetching data
    # (displays if nb.set_opt("verbose", True))
    nb_data_wait("SecurityEvent")
    all_events_df = qry_prov.WindowsSecurity.list_host_events(
        timespan,
        host_name=host_name,
        add_query_items="| where EventID != 4688 and EventID != 4624",
    )

    return all_events_df


# You can add title and/or text to individual functions as they run.
# You can reference text from sections in your YAML file or specify
# it inline (see later example)
@set_text(docs=_CELL_DOCS, key="display_event_timeline")
def _display_event_timeline(acct_event_data):
    # Plot events on a timeline

    # Note the nbdisplay function is a wrapper around IPython.display()
    # However, it honors the "silent" option (global or per-notebooklet)
    # which allows you to suppress output while running.
    return nbdisplay.display_timeline(
        data=acct_event_data,
        group_by="EventID",
        source_columns=["Activity", "Account"],
        legend="right",
    )


# This function has no text output associated with it
def _get_metadata(host_name, timespan):
    return {
        "host": host_name,
        "data_items": {"age": 97, "color": "blue", "country_of_origin": "Norway"},
        "provider": "whois",
        "time_duration": timespan,
    }


# %%
# Extract event details from events
# Note using inline text output here - usually better to store this
# all in the yaml file for maintainability.
@set_text(
    title="Do something else",
    hd_level=3,
    text="""
This may take some time to complete for large numbers of events.

It will do:
- Item one
- Item two
""",
    md=True,
)
def _do_additional_thing(evt_df, event_ids):
    # nb_print is the same as print() except it honors the
    # 'silent' option.
    nb_print("Doing something time-consumingmsticnb.")
    return evt_df[evt_df["EventID"].isin(event_ids)]


---
# More Info

## msticpy
- Documentation - https://msticpy.readthedocs.io
- GitHub - https://github.com/microsoft/msticpy
- PyPI - https://pypi.org/project/msticpy/

## msticnb - Notebooklets
- GitHub - https://github.com/microsoft/msticnb

## Notebooks
- Azure-Sentinel-Notebooks - https://github.com/Azure/Azure-Sentinel-Notebooks
- Binder-able demo - https://github.com/Azure/Azure-Sentinel-Notebooks/tree/master/nbdemo

In [12]:
ts = nbwidgets.QueryTime(units="day")
ts

HTML(value='<h4>Set query time boundaries</h4>')

In [8]:
# %%debug
acc_summary_rslt = acc_summary.run(value="MSTICAdmin", timespan=time_span)

Getting data from AADSignin...


<IPython.core.display.Javascript object>

Getting data from Office365Activity...


<IPython.core.display.Javascript object>

Getting data from Windows Logon activity...


<IPython.core.display.Javascript object>

Getting data from Linux logon activity...


<IPython.core.display.Javascript object>

KeyError: 'Account'

In [11]:
acc_summary.find_additional_data()

Getting data from WindowsSecurity...


<IPython.core.display.Javascript object>

KeyError: "Column 'SourceIP' does not exist!"

--
## Network Flow Notebooklet

In [24]:
nb.init(
    "LocalData",
    LocalData_data_paths=["e:\\src\\msticnb\\msticnb\\tests\\testdata"],
    LocalData_query_paths=["e:\\src\\msticnb\\msticnb\\tests\\testdata"],
)
flow_summary = nb.nblts.azsent.network.NetworkFlowSummary()
flow_result = flow_summary.run(value="MSTICAlertsWin1", timespan=TimeSpan(time_selector=time_span))

e:\src\microsoft\msticpy\msticpy\data\query_store.py:172: UserWarning: e:\src\msticnb\msticnb\tests\testdata\msticpyconfig-test.yaml is not a valid query definition file.
  warnings.warn(f"{file_path} is not a valid query definition file.")


Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

Loading BokehJS ...

Loading BokehJS ...

,source,dest,L7Protocol,FlowDirection,TotalAllowedFlows
0,10.0.3.5,13.107.4.50,http,O,2
1,10.0.3.5,13.65.107.32,https,O,9
2,10.0.3.5,13.67.143.117,https,O,2
3,10.0.3.5,13.71.172.128,https,O,16
4,10.0.3.5,13.71.172.130,https,O,29
5,10.0.3.5,134.170.58.123,https,O,5
6,10.0.3.5,20.38.98.100,https,O,1
7,10.0.3.5,204.79.197.200,https,O,5
8,10.0.3.5,23.48.36.47,http,O,2
9,10.0.3.5,40.124.45.19,https,O,9


Getting data from Whois...
..................

,DestASN,SourceASN,TotalAllowedFlows,L7Protocols,source_ips,dest_ips
0,"AKAMAI-ASN1, EU",No ASN Information for IP type: Private,2.0,[http],[10.0.3.5],[23.48.36.47]
1,"EDGECAST, US",No ASN Information for IP type: Private,6.0,"[https, http]",[10.0.3.5],"[72.21.81.200, 72.21.91.29]"
2,"MICROSOFT-CORP-MSN-AS-BLOCK, US",No ASN Information for IP type: Private,114.0,"[http, https, ntp]",[10.0.3.5],"[13.107.4.50, 13.65.107.32, 13.67.143.117, 13.71.172.128, 13.71.172.130, 134.170.58.123, 20.38.9..."


In [25]:
flow_summary.select_asns()

None does not appear to be an IPv4 or IPv6 address


In [26]:
flow_summary.lookup_ti_for_asn_ips()
flow_summary.show_selected_asn_map()

Getting data from Threat Intelligence...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Warning - query did not complete successfully.
Unknown response from provider: None


AttributeError: 'NetworkFlowResult' object has no attribute 'merge'

In [32]:
from msticnb.nb_metadata import NBMetaData
newmd = eval(repr(nb.nblts.azsent.host.WinHostEvents.metadata))
newmd == nb.nblts.azsent.host.WinHostEvents.metadata

True

In [3]:
nb.nblts.azsent.host.WinHostEvents.import_cell()

AttributeError: type object 'WinHostEvents' has no attribute '__file__'

In [54]:
from msticnb.nb.azsent.host.host_summary import _CELL_DOCS

str(_CELL_DOCS)

metadata_repr = repr(nb.nblts.azsent.host.WinHostEvents.metadata)
metadata_repr = metadata_repr.replace("NBMetaData", "nb_metadata.NBMetaData")

In [4]:
from msticnb.nb.azsent.host import host_summary
host_summary.__file__
with open(host_summary.__file__, "r") as mod_file:
    mod_text = mod_file.read()

repl_text = "_CLS_METADATA, _CELL_DOCS = nb_metadata.read_mod_metadata(__file__, __name__)"
docs_repl = f"_CELL_DOCS = {str(_CELL_DOCS)}\n"
docs_repl = f"{docs_repl}\n_CLS_METADATA = {metadata_repr}"
print(mod_text.replace(repl_text, docs_repl)[1000:3000])

NameError: name '_CELL_DOCS' is not defined

In [6]:
nb.nblts.azsent.host.WinHostEvents.__module__

'msticnb.nb.azsent.host.win_host_events'